In [1]:
# Section 1: Load Setup – Import libraries, connect to DB or define file paths

import pandas as pd
import sqlite3
from pathlib import Path

# SQLite schema as a string (use .format(table_name=...) )
SCHEMA = """
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY,
    customer_name TEXT,
    product TEXT,
    quantity INTEGER,
    unit_price REAL,
    total_price REAL,
    order_date TEXT
);
"""

def load_csv_to_sqlite(csv_path, db_path, table_name):
    """Load a CSV file into a SQLite database table."""
    df = pd.read_csv(csv_path)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(SCHEMA.format(table_name=table_name))
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    return df

def preview_sqlite_table(db_path, table_name, n=5):
    """Preview n rows from a table in a SQLite database."""
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT {n};", conn)
    conn.close()
    return df

# Define file paths
data_dir = Path('.')
csv_full = data_dir / 'transformed_full.csv'
csv_incremental = data_dir / 'transformed_incremental.csv'
loaded_data_dir = data_dir / 'loaded_data'
loaded_data_dir.mkdir(exist_ok=True)

db_full = loaded_data_dir / 'full_data.db'
db_incremental = loaded_data_dir / 'incremental_data.db'


In [2]:
# Section 2: Load Full Transformed Data

df_full = load_csv_to_sqlite(
    csv_path=csv_full,
    db_path=db_full,
    table_name='full_data'
)
print("Full data loaded into SQLite database.")

Full data loaded into SQLite database.


In [3]:
# Section 3: Load Incremental Transformed Data

df_incremental = load_csv_to_sqlite(
    csv_path=csv_incremental,
    db_path=db_incremental,
    table_name='incremental_data'
)
print("Incremental data loaded into SQLite database.")

Incremental data loaded into SQLite database.


In [4]:
# Section 4: Verification (preview loaded tables)

print("\nPreview of full_data (SQLite):")
display(preview_sqlite_table(db_full, 'full_data'))

print("\nPreview of incremental_data (SQLite):")
display(preview_sqlite_table(db_incremental, 'incremental_data'))


Preview of full_data (SQLite):


,id,customer,date,amount,last_updated
0,8435,BestBuy,2025-04-02,1292,2025-04-02T06:41:00
1,3971,Target,2025-04-02,298,2025-04-02T20:41:00
2,5303,eBay,2025-04-02,227,2025-04-02T13:04:00
3,3967,Target,2025-04-02,1115,2025-04-02T16:47:00
4,4883,Costco,2025-04-03,1726,2025-04-03T06:18:00



Preview of incremental_data (SQLite):


,id,customer,date,amount,last_updated
0,8435,BestBuy,2025-04-02,1292,2025-04-02T06:41:00
1,3971,Target,2025-04-02,298,2025-04-02T20:41:00
2,5303,eBay,2025-04-02,227,2025-04-02T13:04:00
3,3967,Target,2025-04-02,1115,2025-04-02T16:47:00
4,4883,Costco,2025-04-03,1726,2025-04-03T06:18:00
